In [14]:
from snowflake.snowpark import Session
from snowflake.snowpark.functions import udf, substr

In [15]:
snowflake_account = 'hz08110.west-europe.azure' 
snowflake_user = 'MLINS'
snowflake_password = 'Mateus2014'
snowflake_user_role = 'SYSADMIN'
snowflake_warehouse = 'DEMO_WH'
snowflake_database = 'DEMO'
snowflake_schema = 'JSONDEMO'

In [16]:
connection_parameters = {
  "account": snowflake_account,
  "user": snowflake_user,
  "password": snowflake_password,
  "role": snowflake_user_role,
  "warehouse": snowflake_warehouse,
  "database": snowflake_database,
  "schema": snowflake_schema
}

In [17]:
session = Session.builder.configs(connection_parameters).create()

In [18]:
print(session.sql("select current_warehouse(), current_database(), current_schema()").collect())

[Row(CURRENT_WAREHOUSE()='DEMO_WH', CURRENT_DATABASE()='DEMO', CURRENT_SCHEMA()='JSONDEMO')]


In [19]:
# session.sql("SELECT RAW_FILE FROM DEMO.JSONDEMO.JSON_RAW").show()

In [20]:
df_nhs_covid = session.sql("SELECT value:areaCode::STRING as area_code," \
                                   "value:areaName::STRING as area_name," \
                                   "value:date::DATE as day_date," \
                                   "value:hospitalCases::INT as num_hospital_cases," \
                                   "value:newAdmissions::INT as num_admissions " \
                           "FROM DEMO.JSONDEMO.JSON_RAW, LATERAL FLATTEN( INPUT => raw_file:body )")

In [21]:
df_nhs_covid.show()

------------------------------------------------------------------------------------
|"AREA_CODE"  |"AREA_NAME"  |"DAY_DATE"  |"NUM_HOSPITAL_CASES"  |"NUM_ADMISSIONS"  |
------------------------------------------------------------------------------------
|E40000003    |London       |2022-07-20  |2303                  |NULL              |
|E40000003    |London       |2022-07-19  |2339                  |NULL              |
|E40000003    |London       |2022-07-18  |2330                  |246               |
|E40000003    |London       |2022-07-17  |2281                  |229               |
|E40000003    |London       |2022-07-16  |2282                  |179               |
|E40000003    |London       |2022-07-15  |2295                  |215               |
|E40000003    |London       |2022-07-14  |2257                  |278               |
|E40000003    |London       |2022-07-13  |2225                  |266               |
|E40000003    |London       |2022-07-12  |2237                  |

In [22]:
df_nhs_covid.write.mode('overwrite').save_as_table('NHS_COVID')

In [23]:
@udf(name="calc_example", is_permanent=True, stage_location="@demo_stage", replace=True, session=session)
def calc_example(x: int, y: int) -> int:
    x = 1 if x is None else x
    y = 1 if y is None else y
    return round(x/y,2)


In [24]:
df_nhs_covid.withColumn('XxX', calc_example(df_nhs_covid.NUM_HOSPITAL_CASES, df_nhs_covid.NUM_ADMISSIONS)).show()

--------------------------------------------------------------------------------------------
|"AREA_CODE"  |"AREA_NAME"  |"DAY_DATE"  |"NUM_HOSPITAL_CASES"  |"NUM_ADMISSIONS"  |"XXX"  |
--------------------------------------------------------------------------------------------
|E40000003    |London       |2022-07-20  |2303                  |NULL              |2303   |
|E40000003    |London       |2022-07-19  |2339                  |NULL              |2339   |
|E40000003    |London       |2022-07-18  |2330                  |246               |9      |
|E40000003    |London       |2022-07-17  |2281                  |229               |9      |
|E40000003    |London       |2022-07-16  |2282                  |179               |12     |
|E40000003    |London       |2022-07-15  |2295                  |215               |10     |
|E40000003    |London       |2022-07-14  |2257                  |278               |8      |
|E40000003    |London       |2022-07-13  |2225                  |266  

In [25]:
df_nhs_covid_month = df_nhs_covid.withColumn('month_year', substr(df_nhs_covid.DAY_DATE, 0, 7))

In [26]:
df_nhs_covid_month.show()

---------------------------------------------------------------------------------------------------
|"AREA_CODE"  |"AREA_NAME"  |"DAY_DATE"  |"NUM_HOSPITAL_CASES"  |"NUM_ADMISSIONS"  |"MONTH_YEAR"  |
---------------------------------------------------------------------------------------------------
|E40000003    |London       |2022-07-20  |2303                  |NULL              |2022-07       |
|E40000003    |London       |2022-07-19  |2339                  |NULL              |2022-07       |
|E40000003    |London       |2022-07-18  |2330                  |246               |2022-07       |
|E40000003    |London       |2022-07-17  |2281                  |229               |2022-07       |
|E40000003    |London       |2022-07-16  |2282                  |179               |2022-07       |
|E40000003    |London       |2022-07-15  |2295                  |215               |2022-07       |
|E40000003    |London       |2022-07-14  |2257                  |278               |2022-07       |


In [ ]:
df_nhs_covid_month.group_by(df_nhs_covid_month.MONTH_YEAR, df_nhs_covid_month.AREA_NAME).agg(sum(df_nhs_covid_month.NUM_HOSPITAL_CASES).alias('h_cases_total')).show()